In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from keras import regularizers
from google.colab import drive
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ZeroPadding2D
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense

In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
datagen_train = ImageDataGenerator(
    rescale=1./255,
      rotation_range=180,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=[0.9, 1.5],
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

In [ ]:
train_dir = "/content/drive/MyDrive/Us/train"
test_dir = "/content/drive/MyDrive/Us/val"

In [ ]:
def create_model():
    
    def add_conv_block(model):
        model.add(Conv2D(96, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(256, 5, activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2))
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(512, 3, activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2))
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(1024, 3, activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(1024, 3, activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(3072, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(4608, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(2, activation='softmax'))
        return model
    
    model = tf.keras.models.Sequential()
    model.add(Input(shape=(56, 56, 3)))
    model = add_conv_block(model)
    

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 16, 16, 256)       0

In [ ]:
input_shape = model.layers[0].output_shape[1:3]
batch_size = 16

In [ ]:
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

Found 45646 images belonging to 2 classes.


In [ ]:
datagen_test = ImageDataGenerator(rescale=1./255)

In [ ]:
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

Found 1194 images belonging to 2 classes.


In [ ]:
h = model.fit(
    generator_train,
    validation_data=generator_test,
    epochs=20, 
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
        tf.keras.callbacks.ModelCheckpoint('models/model_{val_accuracy:.3f}.h5', save_best_only=True,
                                          save_weights_only=False, monitor='val_accuracy')
    ]
)

Epoch 1/20
2853/2853 [==============================] - 17929s 6s/step - loss: 1.3129 - accuracy: 0.5000 - val_loss: 0.8204 - val_accuracy: 0.4841
Epoch 2/20
2853/2853 [==============================] - 719s 252ms/step - loss: 1.2416 - accuracy: 0.4974 - val_loss: 0.8519 - val_accuracy: 0.4849
Epoch 3/20
2853/2853 [==============================] - 720s 252ms/step - loss: 1.2670 - accuracy: 0.4988 - val_loss: 0.9971 - val_accuracy: 0.5142
Epoch 4/20
2853/2853 [==============================] - 719s 252ms/step - loss: 1.2608 - accuracy: 0.5012 - val_loss: 1.9989 - val_accuracy: 0.5168
Epoch 5/20
2853/2853 [==============================] - 734s 257ms/step - loss: 1.1681 - accuracy: 0.5002 - val_loss: 0.7696 - val_accuracy: 0.4841
Epoch 6/20
2853/2853 [==============================] - 745s 261ms/step - loss: 1.1648 - accuracy: 0.4965 - val_loss: 0.9595 - val_accuracy: 0.5034
Epoch 7/20
2853/2853 [==============================] - 742s 260ms/step - loss: 1.2179 - accuracy: 0.5024 - val_l